# House Sales in King County, USA

Predicting house prices using exploratory data analysis and regression models.

This is a cleaned version of the original IBM Skills Network project notebook. It keeps the core analysis and code, and removes platform-specific boilerplate.

## 1. Setup

Import the libraries used throughout the analysis and configure the environment.

In [ ]:
# Suppress non-critical warnings (optional)
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

%matplotlib inline


## 2. Data loading and initial inspection

The dataset `kc_house_data_NaN.csv` is expected to be stored in the local `data/` folder. It contains house sales records for King County, USA (including Seattle).

In [ ]:
DATA_PATH = os.path.join("data", "kc_house_data_NaN.csv")

df = pd.read_csv(DATA_PATH)

# Quick look at the data
print(df.shape)
df.head()


### 2.1 Structure and summary statistics

Inspect column types and basic descriptive statistics to understand the dataset.

In [ ]:
df.info()
df.describe().T


## 3. Data wrangling

We remove identifier columns that are not useful for prediction and handle missing values in key numerical features.

In [ ]:
# Drop identifier columns that do not carry predictive information
for col in ["id", "Unnamed: 0"]:
    if col in df.columns:
        df.drop(col, axis=1, inplace=True)

# Check missing values in bedrooms and bathrooms
print("Missing bedrooms before:", df['bedrooms'].isnull().sum())
print("Missing bathrooms before:", df['bathrooms'].isnull().sum())

# Impute with the mean
df['bedrooms'].fillna(df['bedrooms'].mean(), inplace=True)
df['bathrooms'].fillna(df['bathrooms'].mean(), inplace=True)

print("Missing bedrooms after:", df['bedrooms'].isnull().sum())
print("Missing bathrooms after:", df['bathrooms'].isnull().sum())


## 4. Exploratory data analysis (EDA)

We explore the distribution of key variables and their relationships with the target variable `price`.

In [ ]:
# Distribution of the number of floors
floors_counts = df['floors'].value_counts().to_frame(name='count')
floors_counts


In [ ]:
# Price distribution by waterfront vs. non-waterfront properties
sns.boxplot(x='waterfront', y='price', data=df)
plt.title("House price distribution by waterfront")
plt.show()


In [ ]:
# Relationship between above-ground square footage and price
sns.regplot(x='sqft_above', y='price', data=df)
plt.title("sqft_above vs price")
plt.show()


In [ ]:
# Correlation of numerical features with price
corr_with_price = df.corr(numeric_only=True)['price'].sort_values(ascending=False)
corr_with_price


## 5. Model development

We start with a simple linear regression model, then move to a multivariate model using several predictive features.

In [ ]:
# Simple linear regression using sqft_living as the only predictor
lm_simple = LinearRegression()

X_simple = df[['sqft_living']]
y = df['price']

lm_simple.fit(X_simple, y)
r2_simple = lm_simple.score(X_simple, y)
print(f"R^2 (simple model with sqft_living): {r2_simple:.3f}")


In [ ]:
# Multivariate linear regression with a richer set of features
features = [
    "floors",
    "waterfront",
    "lat",
    "bedrooms",
    "sqft_basement",
    "view",
    "bathrooms",
    "sqft_living15",
    "sqft_above",
    "grade",
    "sqft_living",
]

# Keep only features that actually exist in the dataframe
features = [f for f in features if f in df.columns]

X = df[features]
y = df['price']

lm_multi = LinearRegression()
lm_multi.fit(X, y)

r2_multi = lm_multi.score(X, y)
print("Features used:", features)
print(f"R^2 (multivariate model): {r2_multi:.3f}")


In [ ]:
# Pipeline: scaling + polynomial features + linear regression
poly_pipeline = Pipeline([
    ("scale", StandardScaler()),
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("model", LinearRegression()),
])

cv_scores = cross_val_score(poly_pipeline, X, y, cv=5, scoring="r2")
print("Cross-validated R^2 (polynomial pipeline):")
print("Scores:", cv_scores)
print("Mean R^2:", cv_scores.mean())


## 6. Model evaluation and refinement

We evaluate the model using a train/test split and experiment with a Ridge regression model, with and without polynomial features.

In [ ]:
# Train/test split
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=1
)

print("Number of training samples:", x_train.shape[0])
print("Number of test samples:", x_test.shape[0])


In [ ]:
# Ridge regression on the original feature space
ridge = Ridge(alpha=0.1)
ridge.fit(x_train, y_train)

y_test_pred = ridge.predict(x_test)

r2_ridge = r2_score(y_test, y_test_pred)
rmse_ridge = mean_squared_error(y_test, y_test_pred, squared=False)

print(f"R^2 (Ridge, test set): {r2_ridge:.3f}")
print(f"RMSE (Ridge, test set): {rmse_ridge:,.0f}")


In [ ]:
# Ridge regression with polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)

x_train_pr = poly.fit_transform(x_train)
x_test_pr = poly.transform(x_test)

ridge_poly = Ridge(alpha=0.1)
ridge_poly.fit(x_train_pr, y_train)

y_test_pred_poly = ridge_poly.predict(x_test_pr)

r2_ridge_poly = r2_score(y_test, y_test_pred_poly)
rmse_ridge_poly = mean_squared_error(y_test, y_test_pred_poly, squared=False)

print(f"R^2 (Ridge + polynomial features, test set): {r2_ridge_poly:.3f}")
print(f"RMSE (Ridge + polynomial features, test set): {rmse_ridge_poly:,.0f}")


## 7. Conclusions

- We explored key drivers of house prices in King County, such as square footage, grade, and waterfront.
- A simple linear regression on `sqft_living` provides a baseline model.
- A multivariate model with several features significantly improves the R² score.
- Polynomial features combined with scaling and regularization (Ridge) can further improve performance while controlling overfitting.

This cleaned notebook is intended for portfolio use: it focuses on the analysis and models, without the original platform-specific instructions.